# Multi-agent Collaboration for Financial Analysis

AI agents collaborate with each other to take financial decisions.

Install the given libraries.
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 together==1.2.3

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

## Using Meta-Llama-3.1-405B-Instruct-Turbo Model

In [3]:
from crewai import Agent, Task, Crew

### Using together.ai hosted Open-source LLM

```Python
import os
from together import Together

client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[],
    max_tokens=512,
    temperature=0.7,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1,
    stop=["<|eot_id|>"],
    stream=True
)
print(response.choices[0].message.content)
```

In [4]:
import os
from together import Together

client = Together(api_key='NA')


os.environ["SERPER_API_KEY"] = 'NA'

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        messages=[],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"],
        stream=True
    ),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL. Make sure to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning to predict market movements.

Action: Read website content
A

Final Answer: 
Based on the real-time market data for AAPL from Yahoo Finance, the stock price for Apple Inc. (AAPL) closed at 218.80, showing a slight increase of 0.26%. The stock price has been fluctuating recently, with a 1-year return of 12.32% and a 5-year return of 317.40%. Apple Inc. is a leading consumer electronics company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories globally. The company offers a range of products and services, including iPhone, Mac, iPad, AppleCare support, cloud services, and various subscription services like Apple Music and Apple TV+. With a market capitalization of 3.355T, Apple Inc. has a strong financial position and a solid track record of profitability. It is essential to closely monitor the market movements and trends for AAPL to make informed trading decisions and predict potential opportunities or threats in the market.

> Finished chain.
 

Based on the real-time market data for AAPL

Final Answer: 
Risk tolerance is the degree of risk that an investor is willing to endure given the volatility in the value of an investment. An important component in investing, risk tolerance often determines the type and amount of investments that an individual chooses. Greater risk tolerance is often synonymous with investment in stocks, equity funds, and exchange-traded funds (ETFs), while lower risk tolerance is often associated with the purchase of bonds, bond funds, and income funds.

Understanding Risk Tolerance
All investments involve some degree of risk and knowing their risk tolerance level helps investors plan their entire portfolio, determining how they invest. Based on how much risk they can tolerate, investors are classified as aggressive, moderate, and conservative. Risk tolerance assessments are available online, including risk-related surveys or questionnaires. An investor may also want to review historical returns for different asset classes to determine the volatil

I need to gather detailed execution plans for the approved trading strategies for AAPL to determine the best execution methods considering current market conditions and optimal pricing.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Create detailed execution plans for the approved trading strategies for AAPL considering current market conditions and optimal pricing", "context": "After analyzing the market data for AAPL from Yahoo Finance and considering the user's risk tolerance level, the Trading Strategy Developer has developed a set of potential trading strategies for AAPL that align with the user's risk tolerance."}

> Entering new CrewAgentExecutor chain...
I need to gather detailed information on the approved trading strategies for AAPL considering current market conditions and optimal pricing. I will start by reading the website content to gather specific details on each strategy.

Action: Read website content
Action Input: 

I need to gather detailed analysis of potential risks and suggest mitigation strategies for the proposed trading strategies for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Evaluate risks associated with proposed trading strategies and execution plans for AAPL, and suggest mitigation strategies", "context": "After analyzing the market data for AAPL from Yahoo Finance and considering the user's risk tolerance level, the Trading Strategy Developer has created detailed execution plans for the approved trading strategies for AAPL. These plans take into account the current market conditions and optimal pricing to ensure efficient trade execution and maximize profitability."}


> Entering new CrewAgentExecutor chain...
I need to gather information on the proposed trading strategies and execution plans for AAPL to evaluate the associated risks and suggest mitigation strategies.

Action: Read website content
Action Input: {"website_url": "https:

Final Answer: 
After analyzing the website content from Yahoo Finance regarding AAPL, I have gathered information on Apple Inc.'s stock price, key events, financial highlights, valuation measures, research analysis, and related companies in the industry. Based on this data, I will now evaluate the risks associated with the proposed trading strategies and execution plans for AAPL and suggest mitigation strategies.

> Finished chain.
 

After analyzing the website content from Yahoo Finance regarding AAPL, I have gathered information on Apple Inc.'s stock price, key events, financial highlights, valuation measures, research analysis, and related companies in the industry. Based on this data, I will now evaluate the risks associated with the proposed trading strategies and execution plans for AAPL and suggest mitigation strategies.

Final Answer: 

Risk Analysis Report for AAPL Trading Strategies:

Aggressive Strategy:
- Potential Risks:
  1. High volatility in the market could lead to si

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Aggressive Strategy:
- Potential Risks:
  1. High volatility in the market could lead to significant losses due to the increased exposure to AAPL stocks.
  2. Overreliance on short-term trading opportunities may result in missed long-term growth potential.
- Mitigation Strategies:
  1. Implement stop-loss orders to limit potential losses during market downturns.
  2. Diversify the portfolio with other high-growth stocks to balance out the risk exposure.

Moderate Strategy:
- Potential Risks:
  1. Market fluctuations could impact the value of AAPL stocks and bonds, leading to potential losses.
  2. Inflation and interest rate changes may affect the performance of the bond investments.
- Mitigation Strategies:
  1. Monitor market trends closely and adjust the portfolio allocation accordingly.
  2. Consider hedging strategies to protect against interest rate risks.

Conservative Strategy:
- Potential Risks:
  1. Low returns on low-risk assets may not keep up with inflation, leading to a loss of purchasing power.
  2. Changes in interest rates could impact the performance of bond investments.
- Mitigation Strategies:
  1. Diversify into other income-generating assets such as dividend-paying stocks.
  2. Maintain a cash reserve for liquidity needs and to take advantage of market opportunities.

Overall, it is essential to regularly review and adjust the trading strategies based on market conditions and the user's risk tolerance level to ensure optimal performance and risk management for AAPL investments.